In [ ]:
import numpy as np
import torch

from ssd.utils_ssd300 import dboxes300_coco
from ssd.create_model import nvidia_ssd
from ssd.utils_ssd300 import Encoder
from ssd.model import Loss, SSD300
from ssd.SSD_Transformers import SSDTransformer
from utils.utils import set_seed
from train_model import train_model
from ssd.model_eval import model_evaluate


from torch.utils.data import DataLoader

from config.config import Configs
from ssd.model_eval import model_evaluate

from ssd.decode_results import Processing as processing
from ssd.loader_crowdhuman import CrowdHuman


In [ ]:
device = 'cuda'
ssd_300 = nvidia_ssd(device=device, pretrainded_custom=False, pretrained_default=True)

feat_ext_w = ssd_300.feature_extractor.state_dict()

model = SSD300(label_num=3)
model = model.cuda()

model.feature_extractor.load_state_dict(feat_ext_w)


In [ ]:
batch_size = 32
device = "cuda"

for param in model.parameters():
    param.requires_grad = True

for param in model.feature_extractor.parameters():
    param.requires_grad = False

dboxes = dboxes300_coco()
encoder = Encoder(dboxes)

loss_func = Loss(dboxes, device=device)
transformers_ssd_train = SSDTransformer(dboxes, size=(300, 300), val=False)
transformers_ssd_val = SSDTransformer(dboxes, size=(300, 300), val=True)
set_seed()
data_train = CrowdHuman(
    img_folder="data\\train\\",
    annotate_file="data\\annotation_train.json",
    transform=transformers_ssd_train,
    use_head_bbx=True,
)


train_dataloader = DataLoader(
    data_train,
    batch_size=batch_size,
    shuffle=True 
)


data_val = CrowdHuman(
    img_folder="data\\val\\",
    annotate_file="data\\annotation_val.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

val_dataloader = DataLoader(
    data_val,
    batch_size=batch_size,
    shuffle=False,  
)


data_test = CrowdHuman(
    img_folder="data\\test\\",
    annotate_file="data\\annotation_test.json",
    transform=transformers_ssd_val,
    use_head_bbx=True,
)

test_dataloader = DataLoader(
    data_test,
    batch_size=batch_size,
    shuffle=False,  
)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer=optimizer, patience=4,threshold = 1e-3, verbose=True, factor=0.5, mode = 'max'
)

In [ ]:
train_model(
    model=model,
    encoder = encoder,
    num_epoch=100,
    optimizer=optimizer,
    loss_func = loss_func,
    train_dataloader = train_dataloader,
    val_dataloader = val_dataloader,
    device  = device,
    scheduler=scheduler,
    path_to_save=Configs.path_to_save_state_model,
    use_pick_best=False
)

In [ ]:
res = model_evaluate(
    model = model,
    encoder = encoder,
    val_dataloader = test_dataloader,
    device=device,
    use_pick_best=False,
    threshold = Configs.decode_result["pic_threshold"]
)